# Scripts to Construct a Historical Archive of Contract Status from Publicly Available Sources

As an experiment in making historical data available for analysis, this notebook is developing some basic tools to extract historical contract status data from publicly available pdf files as well as more recently available excel files. The eventual objective is to construct a complete database with this information for the purposes of analysis.

In [160]:
#loading libaries
library(pdftools)
library(readxl)

# Loading a ready Excel File

Note: It looks like this file will need some cleaning up, maybe use a different library to load perhaps the easiest thing to do will be to do a CSV conversion to preserve the dates

In [164]:
excel_contract_status = read_excel("2018-02 Contract Status Report.xlsx")
colnames(excel_contract_status) = excel_contract_status[1, ] # the first row will be the header
excel_contract_status = excel_contract_status[-1,] 
excel_contract_status


CPP #,Contract Package Description,Original Contract Value,Change Orders,Current Contract Value,Incurred,% Exp.,Award Date,Original Completion Date,Revised Completion Date
DB120,West Oahu/Farrington Hwy Guideway,482430201,186423830.73000002,668854031.73000002,665624607.17000008,0.9951717050256137,40128,41568,42797
DB320,Kamehameha Hwy Guideway,372150000,21007573.43,393157573.43000007,391825707.18000001,0.99661238561836529,40724,41927,43008
DB200,Maintenance & Storage Facility,195258000,86517031.730000019,281775031.73000002,281753402.74999928,0.99992324025351731,40724,41993,42553
DB450,Airport Section Guideway/Stations,874750000,14308.330000000002,874764308.33000004,198641165.52999991,0.22707964149705984,42633,44317,44319
DBB171,West Oahu Station Group,56088470,3521866,59610336,26702756.440000005,0.44795514053133345,42205,43171,43171
DBB271,Farrington Highway Station Group,78999000,3473034,82472034,44866668.250000015,0.54402281687389953,42177,43116,43493
DBB371,Kamehameha Hwy Station Group,115805845,1558663,117364508,24162183.080000002,0.20587299765274866,42396,43602,43602
DBB385,Ramp H2R2,5203646,453401,5657047,5310349,0.93871396154212616,42142,42645,43054
DBB505,Airport Section Utilities Cnstr. DBB,27993289.5,182255.74,28175545.240000013,27919489.359999985,0.99091212333891188,41820,42406,42650
DBB525,Airport Section Guideway 7-Pier DBB,3973000,54843,4027843,4027843,1,41900,42063,42118


In [2]:
pdfpages <- pdf_text("19A Contract Status_11.16.pdf")

In [4]:
lines = strsplit(pdfpages, "\n")

# Converting the PDF to a Data Table in R

This process will involve looking at each page of the PDF one at a time and loading in the text from the tables. It's a manual process now, that should be able to be automated to some extent, but for now, exploring the load manually to get a sense for the ranges of values.

The tables are reasonably well laid out for this type of operation. Columns are fixed width, which means subsets are an effective way to extract individual columns from the text read from the PDF. However the width of the column changes from page to page. These width / breakpoints, can probably be scanned for and established at each page in a future utility.

Some of the rows that are read in are the result of a line wrap from a previous row. These will need to be combined at some point.

Finally, the data rows will need to be isolated. This is currently done manually as well, but can potentially be flagged by end markers (the * footnote that focuses on change orders or the "Totals" row on the last page) and if necessary starting markers (the row of headers).

# First Page Conversion

In [83]:
pages_by_lines = strsplit(pdfpages, "\n")
first_page_lines = pages_by_lines[[1]]
data_lines = first_page_lines[9:27]
nchar(data_lines)

[1] 147 147 184 169  60 147 156 183 183 184 157 169 185 169 184 184 184 184 184

In [86]:
contract = substring(data_lines,1,9)
description = substring(data_lines,10,48)
contractor = substring(data_lines,49,84)
base_value = substring(data_lines,85,99)
change_orders = substring(data_lines,100,113)
current_contract_value = substring(data_lines,114,126)
incurred = substring(data_lines,127,139)
percent_expended = substring(data_lines,140,148)
award_date = substring(data_lines,149,160)
completion_date = substring(data_lines,161,171)

tb = data.frame(contract,
                description,
                contractor,
                base_value,
                change_orders,
                current_contract_value, 
                incurred, 
                percent_expended, 
                award_date, 
                completion_date)
tb

contract,description,contractor,base_value,change_orders,current_contract_value,incurred,percent_expended,award_date,completion_date
ART,ART Contracts,Multiple Contracts,"$1,098,500",$0,"$1,098,500","$54,925",5.00,,
CCH-100,Inactive Hart/City CCH,Multiple Contracts,"$15,348,443",$0,"$15,348,443","$14,925,228",97.24,,
CCH-101,HART/ City Dept of BFS,CCH-Budget & Fiscal Services,"$105,092",$0,"$105,092",$0,0.00,7/1/2011,6/30/2016
CCH-102,HART/ City DDC Land Division,CCH-Dept of Design and,"$256,201",$0,"$256,201","$173,182",67.60,7/1/2011,6/30/2016
,,Construction,,,,,,,
CCH-107,HART/ City Corporation Counsel (COR),Multiple Contracts,"$1,616,253","$250,000","$1,866,253","$274,606",14.71,,
CCH-108,HART / Board of Water Supply (BWS),CCH-Board of Water Supply,"$928,325",$0,"$928,325","$928,325",100.00,7/1/2011,
DB-120,West Oahu/Farrington Hwy Guideway DB,Kiewit Infrastructure West Co.,"$482,924,000","$166,584,314","$649,508,314","$630,631,941",97.09,11/11/2009,10/21/2013
DB-200,Maintenance & Storage Facility DB,Kiewit Kobayashi a Joint Venture,"$195,258,000","$79,600,568","$274,858,568","$274,816,663",99.98,6/30/2011,12/20/2014
DB-320,Kamehameha Hwy Guideway DB,Kiewit Infrastructure West Co.,"$372,150,000","$16,928,661","$389,078,661","$337,347,099",86.70,6/30/2011,10/15/2014


# Second Page

In [130]:
second_page_lines = pages_by_lines[[2]]
length(second_page_lines)
nchar(second_page_lines)
data_lines = second_page_lines[9:27]
#data_lines
nchar(data_lines)

contract = substring(data_lines,1,9)
description = substring(data_lines,10,48)
contractor = substring(data_lines,49,84)
base_value = substring(data_lines,85,99)
change_orders = substring(data_lines,100,115)
current_contract_value = substring(data_lines,116,128)
incurred = substring(data_lines,129,141)
percent_expended = substring(data_lines,142,150)
award_date = substring(data_lines,151,160)
completion_date = substring(data_lines,161,172)

tb2 = data.frame(contract,
                description,
                contractor,
                base_value,
                change_orders,
                current_contract_value, 
                incurred, 
                percent_expended, 
                award_date, 
                completion_date)
tb2

[1] 30

[1] 100 108 191 145 186 182 185 148 186 186 171 185 186 186 149 149 171 186 185
[20] 185 172 187 185 158 159 185 170 196 188 176

[1] 186 186 171 185 186 186 149 149 171 186 185 185 172 187 185 158 159 185 170

contract,description,contractor,base_value,change_orders,current_contract_value,incurred,percent_expended,award_date,completion_date
FD-240,Farrington Highway Stations Group FD,URS Corporation,"$9,300,696","$4,994,235","$14,294,931","$13,208,066",92.40,1/12/2011,1/15/2012
FD-340,Kamehameha Hwy Station Group FD,"Anil Verma Associates, Inc.","$8,702,592","$1,257,636","$9,960,228","$9,784,934",98.24,11/16/2012,2/8/2014
FD-430,Airport Sect. Guideway/Util FD,"AECOM Technical Services, Inc.","$38,840,960","$4,293,512","$43,134,472","$41,683,791",96.64,12/22/2011,6/30/2017
FD-440,Airport Station Group FD,"AECOM Technical Services, Inc.","$10,177,365","$1,396,487","$11,573,852","$10,059,820",86.92,11/7/2012,1/27/2014
FD-530,City Center Guideway/Util FD,"AECOM Technical Services, Inc.","$43,948,220","$3,149,342","$47,097,562","$44,431,598",94.34,7/30/2012,4/30/2018
FD-550,SPCD - Dillingham and Kaka'ako SG FD,Perkins+Will,"$18,321,918","$1,309,799","$19,631,717","$12,342,879",62.87,8/8/2013,11/1/2014
HRT-200,HART Labor,Multiple Contracts,"$58,451,942",$0,"$58,451,942","$57,159,803",97.79,,
HRT-201,HART ODC Contracts,Multiple Contracts,"$28,239,699","$216,122","$28,455,821","$26,386,847",92.73,,
MI-900,Fare Collection DFI,INIT Innovations in Transportation,"$15,464,198",$0,"$15,464,198","$1,187,680",7.68,3/18/2016,1/15/2029
MI-930,Elevators & Escalators Install/Maint,Schindler Elevator Corporation,"$50,982,714","$1,069,208","$52,051,922","$6,853,879",13.17,7/31/2013,7/12/2018


# Third Page

In [146]:
third_page_lines = pages_by_lines[[3]]
length(third_page_lines)
nchar(third_page_lines)
data_lines = third_page_lines[9:28]
#data_lines
nchar(data_lines)

contract = substring(data_lines,1,9)
description = substring(data_lines,10,47)
contractor = substring(data_lines,48,84)
base_value = substring(data_lines,85,99)
change_orders = substring(data_lines,100,113)
current_contract_value = substring(data_lines,114,126)
incurred = substring(data_lines,127,141)
percent_expended = substring(data_lines,142,147)
award_date = substring(data_lines,148,158)
completion_date = substring(data_lines,159,172)

tb3 = data.frame(contract,
                description,
                contractor,
                base_value,
                change_orders,
                current_contract_value, 
                incurred, 
                percent_expended, 
                award_date, 
                completion_date)
tb3

[1] 31

[1]  98 106 188 142 183 179 182 145 182 168 168 157  61 155  61 183 168 168 168
[20] 168 167 183 168 168 167 167 168 168 193 185 173

[1] 182 168 168 157  61 155  61 183 168 168 168 168 167 183 168 168 167 167 168
[20] 168

contract,description,contractor,base_value,change_orders,current_contract_value,incurred,percent_expended,award_date,completion_date
MM-920,HDOT Coordination Conslt WOFH,"AECOM Technical Services, Inc.","$3,000,000","$6,500,000","$9,500,000","$7,700,551",81.06,6/7/2011,6/7/2018
MM-921,HDOT Coordination Conslt KHG,"AECOM Technical Services, Inc.","$10,000,000","($1,400,000)","$8,600,000","$4,872,879",56.66,6/29/2012,6/30/2017
MM-922,HDOT Coordination Conslt. Airport,"SSFM International, Inc.","$12,000,000","($5,600,000)","$6,400,000","$3,001,758",46.90,6/8/2012,6/13/2017
MM-925,HDOT Labor - WOFH/KHG Agreements,Hawaii Department of,"$550,000","$2,865,633","$3,415,633","$1,925,497",56.37,10/27/2011,
,,Transportation,,,,,,,
MM-930,HDOT State SOA Manager & Consultant,Hawaii Department of,"$1,272,400","$583,142","$1,855,542","$923,889",49.79,2/3/2012,
,,Transportation,,,,,,,
MM-935,Real Estate Consultant,Paragon Partners Ltd.,"$3,000,000","$5,077,665","$8,077,665","$7,427,051",91.95,3/14/2012,3/13/2017
MM-936,Real Estate Consultant II,Colliers International,"$8,190,000",$0,"$8,190,000",$0,0.00,9/13/2016,9/12/2020
MM-937,Real Estate Consultant - Maps/Surv.,R.M. Towill Corporation,"$2,998,000",$0,"$2,998,000","$1,634,325",54.51,5/22/2014,5/21/2017


# Fourth Page

In [153]:
fourth_page_lines = pages_by_lines[[4]]
length(fourth_page_lines)
nchar(fourth_page_lines)
data_lines = fourth_page_lines[9:18]
total_line = fourth_page_lines[19]
#data_lines
nchar(data_lines)
total_line

contract = substring(data_lines,1,9)
description = substring(data_lines,10,48)
contractor = substring(data_lines,49,84)
base_value = substring(data_lines,85,99)
change_orders = substring(data_lines,100,116)
current_contract_value = substring(data_lines,117,129)
incurred = substring(data_lines,130,146)
percent_expended = substring(data_lines,147,154)
award_date = substring(data_lines,155,164)
completion_date = substring(data_lines,165,176)

tb4 = data.frame(contract,
                description,
                contractor,
                base_value,
                change_orders,
                current_contract_value, 
                incurred, 
                percent_expended, 
                award_date, 
                completion_date)
tb4

[1] 22

[1]  98 106 195 150 190 186 189 153 190 176 190 154 163 164 154 191 154 154 153
[20] 200 192 180

[1] 190 176 190 154 163 164 154 191 154 154

[1] "Total Project:                                                                $3,903,998,079.70   $369,569,216   $4,273,567,295 $2,421,167,243.10   56.65"

contract,description,contractor,base_value,change_orders,current_contract_value,incurred,percent_expended,award_date,completion_date
MM-964,Safety and Security Consultant,Lawson & Associates,"$4,699,573","$232,953","$4,932,526","$4,330,176",87.79,4/23/2014,1/31/2017
MM-970,Fare Collection Consultant,"CH2M Hill, Inc","$1,178,300",$0,"$1,178,300","$117,787",10.00,9/24/2015,12/31/2019
MM-975,LEED Commissioning Services for MSF,"Enovity, Inc.","$278,630","$39,900","$318,530","$254,985",80.05,10/6/2010,1/14/2016
MM-980,Construct. Claims and Litigation Svc,Multiple Contracts,"$810,000",$0,"$810,000","$36,585",4.52,,
MM-981,Complex Real Property Nego. Lit. Sup,Starn O'Toole Marcus & Fisher,"$600,000",$0,"$600,000",$0,0.00,9/9/2016,
MM-982,On Call Appraiser,ACM Consultants Inc,"$1,000,000",$0,"$1,000,000",$0,0.00,6/15/2016,
OTHER,All Project Wide Docs/3P Agreements,Multiple Contracts,"$651,000",$0,"$651,000","$650,645",99.95,,
PA-102,Programmatic Agreement HPC,"Fung Associates, Inc.","$400,000",$0,"$400,000","$197,716",49.43,8/1/2013,5/1/2016
ROW,Real Estate Docs/ 3P Agreements,Multiple Contracts,"$120,885,511",$0,"$120,885,511","$119,380,764",98.76,,
UTIL,Utility Contracts and Agreements,Multiple Contracts,"$95,637,272","($1,920,761)","$93,716,512","$58,421,962",62.34,,


# Final Table Loaded From PDF

Note that this needs some cleaning as well, but generally proves the concept. This table has roughly equivalent information to the excel file above. From this a single function that reads in a PDF file into this data frame format can be created to be totally automated or lightly configured.

In [155]:
contract_status = rbind(tb,tb2,tb3,tb4)
contract_status

contract,description,contractor,base_value,change_orders,current_contract_value,incurred,percent_expended,award_date,completion_date
ART,ART Contracts,Multiple Contracts,"$1,098,500",$0,"$1,098,500","$54,925",5.00,,
CCH-100,Inactive Hart/City CCH,Multiple Contracts,"$15,348,443",$0,"$15,348,443","$14,925,228",97.24,,
CCH-101,HART/ City Dept of BFS,CCH-Budget & Fiscal Services,"$105,092",$0,"$105,092",$0,0.00,7/1/2011,6/30/2016
CCH-102,HART/ City DDC Land Division,CCH-Dept of Design and,"$256,201",$0,"$256,201","$173,182",67.60,7/1/2011,6/30/2016
,,Construction,,,,,,,
CCH-107,HART/ City Corporation Counsel (COR),Multiple Contracts,"$1,616,253","$250,000","$1,866,253","$274,606",14.71,,
CCH-108,HART / Board of Water Supply (BWS),CCH-Board of Water Supply,"$928,325",$0,"$928,325","$928,325",100.00,7/1/2011,
DB-120,West Oahu/Farrington Hwy Guideway DB,Kiewit Infrastructure West Co.,"$482,924,000","$166,584,314","$649,508,314","$630,631,941",97.09,11/11/2009,10/21/2013
DB-200,Maintenance & Storage Facility DB,Kiewit Kobayashi a Joint Venture,"$195,258,000","$79,600,568","$274,858,568","$274,816,663",99.98,6/30/2011,12/20/2014
DB-320,Kamehameha Hwy Guideway DB,Kiewit Infrastructure West Co.,"$372,150,000","$16,928,661","$389,078,661","$337,347,099",86.70,6/30/2011,10/15/2014
